In [1]:
import os
import argparse
import json
import shutil

import numpy as np
import torch
import skvideo.io

from processor.io import IO
import tools
import tools.utils as utils

from IPython.display import clear_output

In [2]:
p = IO()
# clear_output()

Init Start
self.get_parser finish


usage: ipykernel_launcher.py [-w WORK_DIR] [-c CONFIG] [--use_gpu USE_GPU]
                             [--device DEVICE [DEVICE ...]]
                             [--print_log PRINT_LOG] [--save_log SAVE_LOG]
                             [--model MODEL] [--model_args MODEL_ARGS]
                             [--weights WEIGHTS]
                             [--ignore_weights IGNORE_WEIGHTS [IGNORE_WEIGHTS ...]]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/sims/.local/share/jupyter/runtime/kernel-3c35f5f6-0931-45d3-b405-83a49d3e4254.json


SystemExit: 2

/home/sims/anaconda3/envs/gcn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
p.model.eval()

Model(
  (data_bn): BatchNorm1d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (st_gcn_networks): ModuleList(
    (0): st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(3, 192, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(9, 1), stride=(1, 1), padding=(4, 0))
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Dropout(p=0, inplace=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(64, 192, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(9, 1), strid

In [4]:
# video = utils.video.get_video_frames('/media/simslab-cs/A/videoset/cgutest.mp4')
video = utils.video.get_video_frames('/home/sims/2020.mp4')
height, width, _ = video[0].shape

In [5]:
# video_info = utils.openpose.json_pack(output_snippets_dir, video_name, width, height)
#file_json = 'after-filling.json'
file_json = '/home/sims/tf-pose-estimation/2020.json'
with open(file_json, encoding='utf-8') as data_file:
    data = json.loads(data_file.read())
sequence_info = []
weight = data.get('## description').get('image_width')
height = data.get('## description').get('image_height')
for frame_number in data.keys():
    try:
        frame_number = int(frame_number)
        id_name = list(data.get(str(frame_number)).keys())[2]
    except Exception as e:
        continue
    #Get highest possibilites human confiden score --> usualy at detection no 0
    
        
    frame_id = frame_number
    frame_data = {'frame_index': frame_id}
    skeletons = []
    
    score, coordinates = [], []
    skeleton = {}
    
    
    for part in data.get(str(frame_number)).get(id_name):
        coordinates += [int(part.get('position')[0]*weight+0.5),int(part.get('position')[1]*height+0.5)]
        score += [part.get('score')]
    skeleton['pose'] = coordinates
    skeleton['score'] = score
    skeletons +=[skeleton]
    frame_data['skeleton'] = skeletons
    sequence_info += [frame_data]

video_info = dict()
video_info['data'] = sequence_info
video_info['label'] = 'unknowns'
video_info['label_index'] = -1

In [16]:
video_info['data']

[{'frame_index': 25,
  'skeleton': [{'pose': [500,
     626,
     520,
     668,
     0,
     0,
     0,
     0,
     0,
     0,
     527,
     668,
     527,
     793,
     507,
     863,
     527,
     877,
     513,
     1071,
     520,
     1252,
     527,
     877,
     513,
     1071,
     533,
     1266,
     500,
     612,
     500,
     612,
     0,
     0,
     513,
     612],
    'score': [0.20659397542476654,
     0.25242429971694946,
     3.825544807606751e-43,
     3.825544807606751e-43,
     3.825544807606751e-43,
     0.2786552608013153,
     0.23151957988739014,
     0.17128565907478333,
     0.22632208466529846,
     0.15512314438819885,
     0.08265978842973709,
     0.2998135983943939,
     0.18755574524402618,
     0.15206196904182434,
     0.10784979909658432,
     0.23423725366592407,
     3.825544807606751e-43,
     0.23536193370819092]}]},
 {'frame_index': 26,
  'skeleton': [{'pose': [500,
     626,
     520,
     668,
     0,
     64738783,
     0,
     647387

In [39]:
num_person_in=5
num_person_out=1
def video_info_parsing(video_info, num_person_in=5, num_person_out=1):
    data_numpy = np.zeros((3, len(data.keys())-1, 18, num_person_in))
    for frame_info in video_info['data']:
        frame_index = frame_info['frame_index']
        for m, skeleton_info in enumerate(frame_info["skeleton"]):
            if m >= num_person_in:
                break
            pose = skeleton_info['pose']
            score = skeleton_info['score']
            data_numpy[0, frame_index, :, m] = pose[0::2]
            data_numpy[1, frame_index, :, m] = pose[1::2]
            data_numpy[2, frame_index, :, m] = score

    # centralization
    data_numpy[0:2] = data_numpy[0:2] - 0.5
    data_numpy[0][data_numpy[2] == 0] = 0
    data_numpy[1][data_numpy[2] == 0] = 0

    sort_index = (-data_numpy[2, :, :, :].sum(axis=1)).argsort(axis=1)
    for t, s in enumerate(sort_index):
        data_numpy[:, t, :, :] = data_numpy[:, t, :, s].transpose((1, 2,
                                                                    0))
    data_numpy = data_numpy[:, :, :, :num_person_out]

    label = video_info['label_index']
    return data_numpy, label
   

In [35]:
data_numpy[0, 61, :, 0]

array([499.5, 526.5, 746.5, 746.5, 746.5, 526.5, 526.5, 506.5, 526.5,
       512.5, 519.5, 526.5, 519.5, 526.5, 746.5, 506.5, 746.5, 512.5])

In [41]:
#pose, _ = utils.video.video_info_parsing(video_info)
pose, _ = video_info_parsing(video_info)

In [42]:
data = torch.from_numpy(pose)
data = data.unsqueeze(0)
data = data.float().to(p.dev).detach()

In [43]:
output, feature = p.model.extract_feature(data)

In [44]:
output = output[0]
feature = feature[0]
intensity = (feature*feature).sum(dim=0)**0.5
intensity = intensity.cpu().detach().numpy()
label = output.sum(dim=3).sum(dim=2).sum(dim=1).argmax(dim=0)

In [45]:
label_name_path = './resource/kinetics_skeleton/label_name.txt'
with open(label_name_path) as f:
    label_name = f.readlines()
    label_name = [line.rstrip() for line in label_name]
print('Prediction result: {}'.format(label_name[label]))
print('Done.')

Prediction result: mowing lawn
Done.


In [46]:
label = output.sum(dim=3).sum(dim=2).sum(dim=1)
lavel = label.cpu().detach().numpy()
idx = lavel.argsort()[-5:][::-1]

In [47]:
#Other detection 
for i in range(5):
    print('Prediction result: {}'.format(label_name[idx[i]]))

Prediction result: mowing lawn
Prediction result: throwing ball
Prediction result: playing cricket
Prediction result: cleaning pool
Prediction result: changing oil


In [48]:
#Other detection 
for i in range(5):
    print('Prediction result: {}'.format(label_name[idx[i]]))

Prediction result: mowing lawn
Prediction result: throwing ball
Prediction result: playing cricket
Prediction result: cleaning pool
Prediction result: changing oil
